In [ ]:
# data loading config
batch_size = 16
img_height = 224
img_width = 224
dataPath = 'oxford_flowers_102'
labels = 'inferred'
label_mode = 'categorical'  # one hot encoding
color_mode = 'rgb'
shuffle = True
seed = 69
test_split = 0.2  # split into train and test (NOT val), 0-1
AUTOTUNE = tf.data.AUTOTUNE

train_df = pd.read_csv('image_to_label.csv', names=['picture', 'label'])
with open('102_flower_labels.txt') as f:
    regex = re.compile('[^a-zA-Z\s-]')
    classes=[]
    for line in f:
        classes += [regex.sub('', line.replace('\'', '').strip())]


def split_data(dataPath):
    for root, dirs, files in os.walk(dataPath):
        for name in files:
            randomNum = random()
            row = train_df.iloc[train_df.index[train_df['picture'] == name]]
            if randomNum <= test_split:
                os.makedirs('test\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'test\\'+classes[row.label.tolist()[0]-1]+'\\')
            elif test_split< randomNum <= test_split + test_split * (1-test_split):
                os.makedirs('val\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'val\\'+classes[row.label.tolist()[0]-1]+'\\')
            else:
                os.makedirs('train\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'train\\'+classes[row.label.tolist()[0]-1]+'\\')

# split_data(dataPath)  # Only need to run this once

train_datagen = ImageDataGenerator(
    rotation_range=20, width_shift_range=0.2,
    height_shift_range=0.2, brightness_range=(-0.2, 0.2), shear_range=0.2, zoom_range=0.2,
    channel_shift_range=0.2, fill_mode='nearest', horizontal_flip=True, vertical_flip=True)


print('Training data:')
# NOT USING GENERATOR
train_generator = train_datagen.flow_from_directory('train', target_size=(img_height, img_width), batch_size=batch_size,
                                                    color_mode=color_mode, class_mode=label_mode, shuffle=shuffle, seed=seed)
train = keras.preprocessing.image_dataset_from_directory('train', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
train_class_names = train.class_names
print('\nValidation data:')
val = keras.preprocessing.image_dataset_from_directory('val', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
val_class_names = val.class_names
print('\nTesting data:')
test = keras.preprocessing.image_dataset_from_directory('test', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
test_class_names = test.class_names

train = train.cache().prefetch(buffer_size=AUTOTUNE)
val = val.cache().prefetch(buffer_size=AUTOTUNE)
test = test.cache().prefetch(buffer_size=AUTOTUNE)

assert list(train_generator.class_indices.keys()) == train_class_names == val_class_names == test_class_names, 'Classes mismatch!'
classes = list(train_generator.class_indices.keys())
print('\nClasses:', classes)

In [ ]:
import os
import shutil

for i, v in dict.items():
    os.makedirs(v, exist_ok=True)
    shutil.move(os.path.join('train', i), os.path.join(v, i))

In [ ]:
# model
xInput = Input((img_height, img_width, 3), dtype=tf.float16)
efficient_net = tf.keras.applications.efficientnet.EfficientNetB1(include_top=False, weights='imagenet')
x = efficient_net(xInput)
x = Flatten()(x)
x = Dense(256)(x)
x = BatchNormalization(epsilon=1.001e-5)(x)
x = Activation('relu')(x)
xOutput = Dense(len(train_class_names), dtype=tf.float32)(x)  # no activation as loss using logit=True, need use float32 here to ensure model accuracy
model = tf.keras.models.Model(xInput, xOutput)

# compile model

opt = tf.keras.optimizers.Adam(learning_rate=lr)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1)
metrics = ['accuracy']
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=5, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('./best_model',monitor='val_accuracy',save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=3, verbose=1)
]
model.compile(optimizer=opt, loss=loss, metrics=metrics)
model.summary()
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')

In [ ]:
# Training
history = model.fit(train, epochs=epoch, validation_data=val, batch_size=batch_size, verbose=1)

# Evaluate
model.evaluate(test, batch_size=batch_size)